In [ ]:
from franc_lib.semantics import Semantics
from franc_lib.lexical import Preprocessing
import os

In [2]:
from sklearn.externals import joblib
def save_model(data):
    joblib.dump(data, 'saved_model.pkl') 

def load_model():
    return joblib.load('saved_model.pkl')

In [ ]:
s = None
if os.path.exists('saved_model.pkl'):
    s = load_model()
else:
    s = Semantics()
    s.train()
    save_model(data=s)

In [ ]:
s.sentiment_analysis(text="Essa geladeira é ruim")

In [ ]:
s.get_confusion_matrix()
s.